In [453]:
from astrowidgets import ImageWidget
from astrowidgets import ImageWidget
import random
import numpy as np
from astropy.modeling.models import Moffat2D, Gaussian2D
from astropy.convolution import discretize_model
from astropy.table import QTable
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from IPython.display import display
from astrowidgets import ImageWidget

def plot_image(img):
    image = ImageWidget(image_width=500, image_height=500)
    image.load_array(img)
    image.click_center = True
    display(image)
    return image

### Background noise source functions

In [196]:
def readout_noise(output_shape, ron, gain=1, seed=None):
    '''
    Create readout noise pattern image

    Parameters
    ----------
    output_shape : tuple
        Shape of output image
    ron : float
        Readout noise in electrons
    gain : float
        Gain in electrons per ADU
    seed : int
        Seed for random number generator

    Returns
    -------
    output : array_like
        Readout noise pattern
    '''
    noise_rng = np.random.default_rng(seed)
    output = noise_rng.normal(scale=ron/gain, size=output_shape)
    return output

def dark_current(output_shape, dc, exptime=10, gain=1, hot_pixels=0, seed=None):
    '''
    Create dark current pattern image

    Parameters
    ----------
    output_shape : tuple
        Shape of output image
    dc : float
        Dark current in electrons per pixel per second
    exptime : float
        Exposure time in seconds
    gain : float
        Gain in electrons per ADU
    hot_pixels : float
        Fraction of hot pixels
    seed : int
        Seed for random number generator

    Returns
    -------
    output : array_like
        Dark current pattern
    '''

    noise_rng = np.random.default_rng(seed)
    base_current = dc * exptime / gain
    output = noise_rng.poisson(base_current, size=output_shape)

    if hot_pixels:
        y_max, x_max = output_shape
        n_hot = int(hot_pixels * x_max * y_max)
        hot_x = noise_rng.uniform(0, x_max, size=n_hot).astype(int)
        hot_y = noise_rng.uniform(0, y_max, size=n_hot).astype(int)
        hot_val = 10000 * dc * exptime / gain
        output[(hot_y, hot_x)] = hot_val * noise_rng.normal(size=n_hot)

    return output


def sky_background(output_shape, back_level, gain=1, seed=None):
    '''
    Parameters
    ----------
    output_shape : tuple
        Shape of output image
    back_level : float
        Sky background level in ADU per pixel
    gain : float
        Gain in electrons per ADU
    seed : int
        Seed for random number generator

    Returns
    -------
    output : array_like
        Sky background pattern
    '''
    noise_rng = np.random.default_rng(seed)
    output = noise_rng.poisson(back_level*gain, size=output_shape) / gain
    return output

### Star PSFs

In [344]:
class Stars:
    def __init__(self, output_shape, model_params, nstars, seed=None):
        self.nstars = nstars
        self.rng = np.random.default_rng(seed)
        self.border = 5
        self.output_shape = output_shape
        if model_params['type'] == 'moffat':
            self.model = Moffat2D()
            self.alpha, self.beta = self.moffat_fwhm_to_alpha(model_params['fwhm'])
        elif model_params['type'] == 'gaussian':
            self.model = Gaussian2D()
            self.fwhm = model_params['fwhm']

    def make_random_models_table(self, params):
        """
        Make a table of random models.

        Parameters
        ----------
        params : dict
            Dictionary of parameter ranges. Each key is a parameter name,
            and each value is a tuple of (lower, upper, dist), where dist
            is either 'uni' for uniform or 'pow' for power law.
        seed : int, optional
            Random number seed.
        
        Returns
        -------

        table : `~astropy.table.QTable`
            Table of models.
        """
        sources = QTable()
        for param_name, (lower, upper, dist) in params.items():
            if dist == 'uni': sources[param_name] = self.rng.uniform(lower, upper, self.nstars)
            elif dist == 'pow': sources[param_name] = upper + lower - self.rng.power(50, self.nstars) * upper
        return sources

    def make_model_sources_image(self, source_table, oversample=1):
        """
        Make an image of model sources.

        Parameters
        ----------
        output_shape : tuple
            Shape of output image.
        model : `~astropy.modeling.Model`
            Model to use for sources.
        source_table : `~astropy.table.QTable`
            Table of source parameters.
        oversample : int, optional
            Oversampling factor for discretizing the model.
        
        Returns
        -------
        image : `~numpy.ndarray`
            Image of model sources.
        flux : list
            List of fluxes of each source.
        """

        image = np.zeros(self.output_shape, dtype=float)
        yidx, xidx = np.indices(self.output_shape)

        params_to_set = []
        for param in source_table.colnames:
            if param in self.model.param_names:
                params_to_set.append(param)
        init_params = {param: getattr(self.model, param) for param in params_to_set}

        try:
            flux = []
            for source in source_table:
                for param in params_to_set:
                    setattr(self.model, param, source[param])
                if oversample == 1:
                    s = self.model(xidx, yidx)
                    image += s
                    flux.append(np.sum(s))
                else:
                    image += discretize_model(self.model, (0, self.output_shape[1]),
                                            (0, self.output_shape[0]), mode='oversample',
                                            factor=oversample)
        finally:
            for param, value in init_params.items():
                setattr(self.model, param, value)

        return image, flux
    

    def create_stars(self, amp_range, ZP=21):
        """
        Create an image of stars and a table of source parameters.

        Parameters
        ----------
        amp_range : tuple
            Range of amplitude of sources.

        Returns
        -------
        data : `~numpy.ndarray`
            Image of sources.
        sources : `~astropy.table.QTable`
            Table of source parameters.
        """

        param_ranges = {'amplitude': [amp_range[0], amp_range[1], 'pow'],
                        'x_0': [self.border, self.output_shape[1]-self.border, 'uni'],
                        'y_0': [self.border, self.output_shape[0]-self.border, 'uni'],
                        'alpha': [self.beta, self.beta, 'uni'],
                        'gamma': [self.alpha, self.alpha, 'uni']}

        sources = self.make_random_models_table(param_ranges)
        data, flux = self.make_model_sources_image(sources)
        sources['flux'] = flux
        sources['fwhm'] = self.moffat_alpha_to_fwhm(sources['alpha'])
        sources['mag'] = ZP - 2.5 * np.log10(flux)
        sources.rename_column('alpha', 'beta')
        sources.rename_column('gamma', 'alpha')
        return data, sources

    @staticmethod
    def moffat_alpha_to_fwhm(alpha, beta=4.765):
        fwhm = 2 * alpha * np.sqrt(2 ** (1/beta) - 1)
        return fwhm
    
    @staticmethod
    def moffat_fwhm_to_alpha(fwhm, beta=4.765):
        alpha = fwhm / (2 * np.sqrt(2**(1/beta) - 1)) 
        return alpha, beta



### Create synthetic image

In [450]:
class SyntImage:
    
    def __init__(self, output_shape, back_level, ron, dc, hot_px, gain,
                 exptime, mag_range, ZP, fwhm, nstars, model, seed = None):
        """
        Parameters
        ----------
        output_shape : tuple
            Shape of the output image.
        back_level : float
            Background level in ADU.
        ron : float
            Readout noise in electrons.
        dc : float
            Dark current in electrons per second per pixel.
        hot_px : float
            Fraction of hot pixels.
        gain : float
            Gain in electrons per ADU.
        exptime : float
            Exposure time in seconds.
        mag_range : tuple
            Magnitude range of the stars.
        ZP : float
            Zero point.
        fwhm : float
            FWHM of the stars in pixels.
        nstars : int
            Number of stars.
        model : str
            Model of the star PSF.
        seed : int
            Seed for the random number generator.
        """

        self.output_shape = output_shape
        self.back_level = back_level
        self.ron = ron
        self.dc = dc
        self.hot_px = hot_px
        self.gain = gain
        self.exptime = exptime
        self.mag_range = mag_range
        self.ZP = ZP
        self.fwhm = fwhm
        self.nstars = nstars
        self.model = model
        self.seed = seed

    def create_data(self):
        """
        Create a synthetic image with stars and background noise sources.
        
        Returns
        -------
        image : array
            Output image.
        sources : array
            Array of sources.
        """

        image = np.zeros(list(self.output_shape))

        # Add background noise sources
        image += readout_noise(self.output_shape, self.ron, self.gain, self.seed)
        image += dark_current(self.output_shape, self.dc, self.exptime, self.gain, self.hot_px, self.seed)
        image += sky_background(self.output_shape, self.back_level, self.gain, self.seed)

        # Add stars
        model_params = {'type': self.model,
                        'fwhm': self.fwhm}
        stars = Stars(self.output_shape, model_params, self.nstars, self.seed)
        data, sources = stars.create_stars(self.mag_range, self.ZP)
        image += data

        return image.astype(np.float32), sources

    def create_header(self):
        """
        Create a header with the image parameters.
        
        Returns
        -------
        header : dict
            Header with the image parameters.
        """
        header = fits.header.Header()
        header['NAXIS'] = 2
        header['NAXIS1'] = self.output_shape[0]
        header['NAXIS2'] = self.output_shape[1]
        header['GAIN'] = (self.gain, 'Gain in electrons per ADU')
        header['EXPTIME'] = (self.exptime, 'Exposure time in seconds')
        header['RON'] = (self.ron, 'Readout noise in electrons')
        header['DC'] = (self.dc, 'Dark current in electrons per second per pixel')
        header['SKYFLUX'] = self.back_level
        header['SKYMAG'] = self.ZP - 2.5 * np.log10(self.back_level)
        header['HOTPX'] = (self.hot_px, 'Fraction of hot pixels')
        header['ZP'] = (self.ZP, 'Zero point')
        header['FWHM'] = (self.fwhm, 'FWHM of the stars in pixels')
        header['NSTARS'] = (self.nstars, 'Number of stars')
        header['MAGMIN'] = (self.mag_range[0], 'Minimum star magnitude')
        header['MAGMAX'] = (self.mag_range[1], 'Maximum star magnitude')
        header['MODEL'] = (self.model, 'Model of the star PSF')

        return header
    
    def create_fits(self):
        data, sources = self.create_data()
        header = self.create_header()
        hdu = fits.PrimaryHDU(data, header=header)
        return hdu, sources.to_pandas()

## Radial degradation

In [ ]:
def fwhm_degradation (pos, center, size, fwhm_0, f = 3):
    center = np.array(center).astype(int)
    dist = np.sqrt(np.sum((np.array(pos) - center)**2))
    fwhm = fwhm_0 * (1 + (f-1) * (2*dist/size))
    return fwhm

def snr_degradation (pos, center, size, snr_0, f = 2):
    center = np.array(center).astype(int)
    dist = np.sqrt(np.sum((np.array(pos) - center)**2))
    snr = snr_0 * (1 + np.e**((f-1) * (2*((size/2)-dist)/size)))
    return snr

def sky_degradation (pos, center, size, sky_0, f = 2):
    center = np.array(center).astype(int)
    dist = np.sqrt(np.sum((np.array(pos) - center)**2))
    sky = sky_0 * (1 + (f-1) * (2*((size/2)-dist)/size))
    return sky

## **Example:** single image

In [451]:
# Parameters
output_shape = (1000, 1000)
back_level = 100
ron = 3
dc = 0.01
hot_px = 0.00001
gain = 0.44
exptime = 30
amp_range = (10, 5000)
ZP = 27
fwhm = 5
model = 'moffat'
nstars=300

# Create fits
si = SyntImage(output_shape, back_level, ron, dc, hot_px, gain, exptime, amp_range, ZP, fwhm, nstars, model)
hdu, sources = si.create_fits()

hdu.writeto('syn_ima_2.fits', overwrite=True)
sources.to_csv('syn_ima_sources_2.csv', index=False)

In [454]:
img = plot_image(hdu.data)

ImageWidget(children=(Image(value=b'', format='jpeg', height='500', layout="Layout(margin='0')", width='500'),…